In [39]:
import numpy as np
import scipy
from scipy import stats
import pylab as plt
%matplotlib inline

EPSILON=0.5

# Datum
trainX = np.loadtxt('X_train.csv', delimiter=',', skiprows=1)
traint = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)
traint = traint[:,1][:,None]-1

"""def cv(trainX, traint, w, folds=20):
    slice_size = int(trainX.shape[0]/folds)
    total_accuracy = 0
    
    for i in range(folds):
        split_start = slice_size * i
        split_end = slice_size * (i+1)
        cv_training_X = np.concatenate((trainX[:split_start], trainX[split_end:]))
        cv_training_t = np.concatenate((traint[:split_start], traint[split_end:]))
        cv_testing_X = trainX[split_start:split_end]
        cv_testing_t = traint[split_start:split_end]
        
        hits = 0
        for i in range(cv_testing_X.shape[0]):
            prediction = to_binary(h(cv_testing_X[i], w))
            diff = prediction - cv_testing_t[i][0]
            if diff == 0:
                hits += 1
        accuracy = hits/slice_size
        
        total_accuracy += accuracy
        print("CV iteration and accuracy:", i, accuracy)
        
    total_accuracy /= folds
    return total_accuracy"""

"""def H(X, w):
    t = np.zeros((X.shape[0],1))
    for i in range(X.shape[0]):
        print("Single term:", h(X[i], w))
        t[i,0] = to_binary(h(X[i], w))
    return t"""
    
def to_binary(x):
    if x < EPSILON:
        return 0
    return 1

def h(x, w):
    return sigmoid(np.dot(x, w))

def sigmoid(x):
    val = scipy.special.expit(x)
    return val

def filter_by_distinction(trainX, traint, n=10):
    ps = np.zeros(trainX.shape[1])
    
    for i in range(trainX.shape[1]):
        feature = trainX.T[i][:,None]
        label = traint.flatten()[:,None]
        
        stacked = np.hstack((feature, label))
        stacked.sort(axis=0)
        count = (stacked[:,1] == 1).sum()
        group1 = stacked[:count,0]
        group2 = stacked[count:,0]
        
        p = stats.ttest_ind(group1,group2)[1]
        ps[i] = p
        
    features_by_distinction = ps.argsort()[:n]
    #print(features_by_distinction)
    newX = np.zeros((trainX.shape[0], n))
    
    for i in range(n):
        newX[:,i] = trainX[:,features_by_distinction[i]]
    
    return newX, features_by_distinction

"""def single_prediction_cost(prediction, actual):
    if actual == 1:
        return -np.log(prediction)
    return -np.log(1-prediction)"""

"""def total_cost(X, t, w):
    m = X.shape[0]
    c = 0
    for i in range(m):
        c += single_prediction_cost(h(X[i], w), t[i])
    return c/m"""

"""def single_weight_change(weight, w_number, X, t, alpha=1):
    m = X.shape[0]
    summ = 0
    
    for i in range(m):
        inp = X[i][:,None]
        #print(inp)
        #print(w)
        prediction = h(inp, w)
        #print(prediction)
        actual = t[i]
        summ += (prediction-actual) * X[i,w_number]
    
    return -alpha*summ/m"""

def plot_pair(X, t, f1, f2):
    plt.figure()
    pos0 = np.where(t==0)[0]
    pos1 = np.where(t==1)[0]
    plt.plot(X[pos1,f1], X[pos1,f2],'bo')
    plt.plot(X[pos0,f1], X[pos0,f2],'ro')
    
def save_predictions(predictions):
    N = predictions.shape[0]
    
    output = np.ones((N, 2))
    output[:,0] = range(N)
    output[:,1] = predictions.ravel()
    np.savetxt('pred_classification.csv', output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")

def get_optimal_weights(w, X, t, iterations=300, alpha=0.002):
    sample_count = X.shape[0]
    weight_count = X.shape[1]
    
    for iters in range(iterations):
        for j in range(weight_count):
            change_sum = 0
            for i in range(sample_count):
                prediction = to_binary(h(X[i,:], w))
                change = (prediction-t[i])*X[i,j]
                change_sum += change
                
            w[j] = w[j] - alpha*change_sum
        
        print(w[0:5].T)
        #print("Updated weights after iteration", iters)
        
    return w

#plot_pair(trainX, traint, 71, 4)

trainX, top_feature_indices = filter_by_distinction(trainX, traint, 10)
w = np.zeros(trainX.shape[1])[:,None]
w = get_optimal_weights(w, trainX, traint)
print("Final weights:", w)

#cv(np.dot(trainX, w), traint)

# Keep only top features
testX = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)
testX = testX[:,top_feature_indices]

preds = np.zeros((testX.shape[0], 1))
for i in range(testX.shape[0]):
    preds[i] = to_binary(h(testX[i], w))
    print(preds[i,0])
save_predictions(preds+1)

[[-0.21437527  0.1559461   0.07659667 -0.03420856  0.01132202]]
[[-0.10558227  0.34062586  0.36151733 -0.00116871  0.61589141]]
[[-0.31427079  0.22815035  0.23021933 -0.13023462  0.42798957]]
[[-0.52864606  0.07248803  0.0343991  -0.2622465   0.09486532]]
[[-0.50661943  0.05176438  0.0331737  -0.26973848  0.09513194]]
[[-0.4696283   0.05152029  0.05165932 -0.27050176  0.13390221]]
[[-0.42722718  0.05656239  0.07722649 -0.26891576  0.18719842]]
[[-0.3473102   0.14056217  0.19618077 -0.24706669  0.434756  ]]
[[-0.30438781  0.28868569  0.42642715 -0.25084606  0.95071591]]
[[-0.51505222  0.15646585  0.2669376  -0.38066265  0.69921861]]
[[-0.72942748  0.00080353  0.07111737 -0.51267452  0.36609436]]
[[-0.70674154 -0.01865838  0.0714243  -0.52001137  0.36963609]]
[[-0.67824646 -0.0286895   0.07980673 -0.52459183  0.38924355]]
[[-0.64143353 -0.02929115  0.09719637 -0.52628528  0.42642717]]
[[-0.59586925 -0.01657055  0.12979907 -0.52275572  0.49429563]]
[[-0.52120989  0.07650128  0.26703189 -0

[[-3.02153283 -1.11727229  0.8067106  -4.0991829   4.67477084]]
[[-3.09867278 -1.17896814  0.7387608  -4.15362656  4.57129924]]
[[-3.29904277 -1.32420147  0.55562558 -4.27619357  4.25664191]]
[[-3.26953333 -1.33563248  0.56188246 -4.27810009  4.26919183]]
[[-3.23017206 -1.3376379   0.57971486 -4.27498141  4.30407566]]
[[-3.18748411 -1.33693194  0.60163687 -4.27073656  4.34711295]]
[[-3.13548507 -1.32456484  0.63613931 -4.261148    4.41397628]]
[[-3.03807326 -1.24541102  0.75691337 -4.23128951  4.65364968]]
[[-2.97645319 -1.08784543  1.00275438 -4.22451798  5.19616548]]
[[-3.12940355 -1.07992559  1.03876673 -4.32924591  5.35782846]]
[[-3.34377882 -1.23558791  0.8429465  -4.46125779  5.02470422]]
[[-3.28058448 -1.15356133  0.96262603 -4.45056456  5.28466621]]
[[-3.26493006 -1.04385248  1.13052973 -4.46724014  5.67735199]]
[[-3.4510573  -1.1421384   1.00916425 -4.5858248   5.50340915]]
[[-3.66343256 -1.29769704  0.81427552 -4.71783668  5.17183228]]
[[-3.64062526 -1.31720873  0.81456    -4

[[-5.70503035 -2.38117662  1.53969813 -8.06756044  9.25585831]]
[[-5.66928797 -2.38566431  1.55520325 -8.06742549  9.28796007]]
[[-5.62952497 -2.38533067  1.57557384 -8.0656443   9.32968033]]
[[-5.58031937 -2.37548823  1.60739876 -8.05881824  9.39356308]]
[[-5.46438611 -2.19186883  1.8891542  -8.01952494  9.98484469]]
[[-5.55809523 -2.13901076  1.98302984 -8.09548307 10.25363438]]
[[-5.77166781 -2.2928069   1.78944503 -8.22715861  9.92538427]]
[[-5.97255335 -2.4403655   1.60519773 -8.35103977  9.61100274]]
[[-5.94145426 -2.44753657  1.61627141 -8.35395342  9.635238  ]]
[[-5.91295918 -2.45756769  1.62465385 -8.35853388  9.65484545]]
[[-5.88292278 -2.46609936  1.63468079 -8.36263421  9.67764817]]
[[-5.84945188 -2.469766    1.64878642 -8.3653412   9.70841604]]
[[-5.80534357 -2.45868051  1.6773826  -8.36255588  9.76788798]]
[[-5.74092788 -2.39101095  1.77069463 -8.34756451  9.96570721]]
[[-5.64761334 -2.21270895  2.04484989 -8.32019984 10.55330052]]
[[-5.85984146 -2.35549672  1.8664785  -8

In [ ]:
# sklearn attemp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

split = 600
X_train = trainX[:split]
t_train = traint[:split].ravel()

model = clf.fit(X_train, t_train)
predictions = model.predict(X_test)+1

N = X_test.shape[0]
output = np.ones((N, 2))
output[:,0] = range(N)
output[:,1] = predictions
np.savetxt('pred_classification.csv', output, fmt='%d', delimiter=",", header="Id,EpiOrStroma")